# Lesson 4
## New .env variables
you need to add three new variables to you r .env file, if you didnt have done so yet

```env
HOSPITAL_AGENT_MODEL=gpt-3.5-turbo-1106
HOSPITAL_CYPHER_MODEL=gpt-3.5-turbo-1106
HOSPITAL_QA_MODEL=gpt-3.5-turbo-0125
```
Your .env file now includes variables that specify which LLM you’ll use for different components of your chatbot. You’ve specified these models as environment variables so that you can easily switch between different OpenAI models without changing any code. Keep in mind, however, that each LLM might benefit from a unique prompting strategy, so you might need to modify your prompts if you plan on using a different suite of LLMs.

## Create a Neo4j Vector Chain
### Vector Search Indexes
Vector databases are specifically designed to store and query high-dimensional vectors, which are numerical representations of unstructured data like text, images, or audio. Vector search indexes enable efficient similarity searches (semantic searches) based on that semantically similar objects are represented by vectors that are close to each other in the vector space.

Neo4j, a graph database, has added vector search capabilities as an add-on. Neo4j's [Vector search indexes](https://neo4j.com/docs/cypher-manual/current/indexes-for-vector-search/) allows developers to create vector indexes. These indexes work with node or relationship properties that contain vector embeddings

### Creating the embeddings
In LangChain, you can use Neo4jVector to create review embeddings and returns a "vector store" to query it. Here’s the code to create the indexes.

The method takes several parameters:
- embedding: an instance of OpenAIEmbeddings, which will be used to generate embeddings for the text data. The specific embedding model used will depend on the requirements of your project, such as the type of text data, the desired level of accuracy, and the computational resources available. We use it as we have openai api key but there are others in the langchain library.
    - HuggingFaceEmbeddings: uses Hugging Face's Transformers library to generate embeddings
    - SentenceTransformersEmbeddings: uses the Sentence Transformers library to generate embeddings
    - CustomEmbeddings: a custom implementation of the Embeddings interface, allowing you to use your own embedding model
    - Word2VecEmbeddings
    - GloVeEmbeddings
    - FastTextEmbeddings
    - BERTEmbeddings
- url, username, password: these are used to connect to the Neo4j database, and are retrieved from environment variables (NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD).
- index_name: the name of the vector index to create ("reviews").
- node_label: the label of the nodes in the graph database that will be indexed ("Review").
- text_node_properties: a list of properties on the nodes that contain text data to be indexed (["physician_name", "patient_name", "text", "hospital_name"]).
- embedding_node_property: the property on the nodes where the generated embeddings will be stored ("embedding").

---
> ⚠️ **Note:** `from_existing_graph` is primarily used when your text data is already in the Neo4j graph, but the embeddings and the vector index might not yet exist or need to be generated by the LangChain process. It seems to automate querying Neo4j to get the relevant text data, calculate embeddings for that text, storing those newly generated embeddings back into the specified node property in Neo4j and creating the necessary vector index if it doesn't exist. Therefore, if the embeddings and the corresponding vector index already fully exist and are populated in your Neo4j database, the method you would typically use is `Neo4jVector.from_existing_index`, not `from_existing_graph`
---



In [ ]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings


neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

In [2]:
neo4j_vector_index

If you filled the database in the previous lesson, after running this you will see that the embeddings had been created as an index of the review text 

![embeddings.png](embeddings.png)

This allows you to answer questions like "Which hospitals have had positive reviews?" It also allows the LLM to tell you which patient and physician wrote reviews matching your question.

### The retriever
To use the Retrieval Augmented Generation we need to create a retriever based in this vector store we just created.

In [3]:

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
HOSPITAL_QA_MODEL = os.getenv("HOSPITAL_QA_MODEL") # gpt-3.5-turbo-0125

# chat prompt template with system and human messages
review_template = """Your job is to use patient
reviews to answer questions about their experience at a hospital. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer, say you don't know.
{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=review_template)
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

# the document chain, based in our vector index
reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=HOSPITAL_QA_MODEL, temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)

# the complete review chain
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

The last part is setting up a RetrievalQA chain, what is called a document chain.

**The retriever/document chain**

* `reviews_vector_chain` is the instance of `RetrievalQA`
* The `from_chain_type` defines the chain type (pass all the docs found entirely, pass only relevant chunks of it, ...). In this case, the chain type is `"stuff"` the standard one, it passes all retrieved data as context. Different chain types are "stuff", "map refine", and "rerank", with the later are alternative complex ones to avoid overflowing the context window 
* The `llm` parameter specifies the language model that will be used specifically for the question-answering system and its temperature.
* The `retriever` parameter specifies the vector store (retriever) that will be used to fish the relevant documents from the database. In this case, it's an instance of `neo4j_vector_index.as_retriever(k=12)`, which is a retriever that uses the Neo4j vector index to gather documents. The `k` parameter specifies the number of documents to retrieve (12).

**Combining the documents chain and LLM chain**

* The `combine_documents_chain` method is used to combine the documents chain (which retrieves relevant documents from the database) with the LLM chain (which uses the language model to generate answers).
* The `llm_chain.prompt` parameter specifies the prompt that will be used for the LLM chain. In this case, it's set to the `review_prompt` instance that was defined earlier.


In [5]:
query = """What have patients said about hospital efficiency?
        Mention details from specific reviews."""

response = reviews_vector_chain.invoke(query)
response.get("result")

Patients have generally praised the efficiency of the hospital staff and the cleanliness of the facilities. For example, Kevin Cox mentioned that the hospital staff at Wallace-Hamilton was efficient, and Jesse Marquez also noted that the hospital provided excellent medical care at the same facility. Additionally, Kim Franklin commented on the efficiency of the hospital staff and the cleanliness of the facilities at Wallace-Hamilton.
